# Overlapping records in two datasets

In [1]:
import numpy as np
import pandas as pd
import jellyfish
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
pd.set_option('display.width', 120)
pd.set_option('display.max_columns', 40)
pd.set_option('display.max_rows', 100)

# Read datasets to match

Read datasets A and B into Pandas data frames A and B, respectively.

In [3]:
df_a = pd.read_csv('datasetA.csv', dtype=str)
df_a

,first_name,last_name,street_number,address_1,address_2,town,zipcode,state,date_of_birth,age,phone_number,soc_sec_id
0,lachlan,carmody,19,cade place,b,hamilton,4556,qld,19960616,31.0,02 92812617,2231206
1,sarah,tippins,3,keverstone circuit,NaN,woodpark,5540,qld,19280526,13.0,04 11695119,4612010
2,jessica,cheshire,12,halfrey circuit,NaN,jesmond,5172,nsw,19210205,26.0,02 02242599,3144617
3,natalia,tiller,73,eagle circuit,NaN,camden,5114,nt,19351203,33.0,07 91923807,1245778
4,lily,blackwell,18,bindel street,cambooya,oak flats,6230,nsw,19791106,NaN,04 45361517,2195128
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,jessica,clarke,52,NaN,msf 60,chelmsford,2566,nsw,19601129,NaN,03 40798285,4849937
9996,alessandra,moody,514,withnell circuit,NaN,narrogin,6207,vic,19210819,NaN,03 44448442,3068581
9997,darcie,burford,4,longerenong street,cambridge park,ballarat,NaN,nsw,19970902,33.0,04 31228862,9011666
9998,steven,white,70,investigator street,NaN,parkdale,2640,nsw,19280106,34.0,04 90977051,1490648


In [4]:
df_b = pd.read_csv('datasetB.csv', dtype=str)
df_b

,first_name,last_name,street_number,address_1,address_2,town,zipcode,state,date_of_birth,age,phone_number,soc_sec_id
0,baysen,scudds,11,helemontstreet,NaN,picton,4055,vic,19340808,29.0,03 82765568,1070893
1,jsohua,greej,30,muecke place,rockview,willetton,6023,wa,19164909,NaN,02 30027583,5782396
2,alicia,treumekr,24,hambidge crescent,NaN,bligh park,4216,qld,19000819,21.0,08 11540291,7788784
3,anna,reid,13,creswellsstreet,oxonia,parksie,4005,vim,19807429,NaN,03 26039045,1646214
4,dato,natasha,96,de burgh street,NaN,bayswater,4670,vic,19301075,23.0,08 76809985,4308494
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,ema,britten,4,mugga fay,NaN,north beach,2211,ws,19150617,42.0,02 42401723,9558954
9996,keegan,herbdrt,56,dunbar sxtreet,dutch care hostel,como,4065,qld,NaN,30.0,NaN,8854365
9997,makenzie,huxldey,6,frencham place,NaN,longrgeach,5271,wa,19810514,29.0,07 53146771,4011126
9998,sybella,irizsrry,5,mckinlay street,NaN,vermont,3141,qld,19890909,35.0,08 92355273,7620274


# Examples of matches

In [5]:
df_a.loc[[962, 2847, 7787, 6724]]

,first_name,last_name,street_number,address_1,address_2,town,zipcode,state,date_of_birth,age,phone_number,soc_sec_id
962,NaN,fleet,65,dartnell street,NaN,reynella,4215,nsw,19160720,23.0,02 46529999,1933859
2847,cameron,lademan,19,ebden street,NaN,williamstown,3165,nsw,19430515,29.0,08 39803141,8805927
7787,jamie,caruana,18,packham place,NaN,sunshine north,2096,nsw,19590202,23.0,08 71094666,5726002
6724,sophie,haupt,6,macrossan crescent,NaN,kyogle,2534,nsw,19071127,33.0,08 51432405,8085321


In [6]:
df_b.loc[[2704, 6268, 4843, 7373]]

,first_name,last_name,street_number,address_1,address_2,town,zipcode,state,date_of_birth,age,phone_number,soc_sec_id
2704,NaN,fleent,65,street datnell,NaN,reynella,4215,nsw,19160720,23.0,02 45629999,1933859
6268,camern,lademan,199,ebden street,NaN,bronte,3615,nsw,19430515,29.0,08 39803141,8805927
4843,jamie,caruana,18,packhamplace,NaN,sunshine north,2069,nzw,19590202,23.0,08 12512838,5726002
7373,sophie,haupt,6,macrossan yescent,NaN,kyogle,2534,nsw,NaN,33.0,08 51432405,4114985


# Exact matches

Merging the data frames by requiring one or more exact matches **does
not find all overlapping records**, since it does not take into account
differences between the values of the match columns in data frame A
and data frame B.

In [7]:
EXACT_COLS = ['last_name', 'date_of_birth', 'address_1']

In [8]:
df_a[EXACT_COLS].notnull().all(axis=1).sum()

8472

In [9]:
df_b[EXACT_COLS].notnull().all(axis=1).sum()

7824

Merge data frames A and B on specified columns for exact matching

In [10]:
exact_merge = df_a[df_a[EXACT_COLS].notnull().all(axis=1)]\
                  .reset_index(drop=False)\
                  .merge(df_b[df_b[EXACT_COLS].notnull().all(axis=1)].reset_index(drop=False),
                         how='inner', on=EXACT_COLS, suffixes=('_a', '_b'))\
                  .set_index(['index_a', 'index_b'] + EXACT_COLS, drop=True)
exact_merge = exact_merge[sorted(exact_merge.columns)]
exact_merge

,,,,,address_2_a,address_2_b,age_a,age_b,first_name_a,first_name_b,phone_number_a,phone_number_b,soc_sec_id_a,soc_sec_id_b,state_a,state_b,street_number_a,street_number_b,town_a,town_b,zipcode_a,zipcode_b
index_a,index_b,last_name,date_of_birth,address_1,,,,,,,,,,,,,,,,,,
4,7442,blackwell,19791106,bindel street,cambooya,cambooya,NaN,NaN,lily,liplt,04 45361517,04 45361517,2195128,2195128,nsw,nsw,18,1,oak flats,oak flats,6230,6230
11,1897,hyland,19770727,badimara street,NaN,lst hse on left,32.0,8.0,talissa,talsxa,07 50826244,07 50826244,8476381,8476381,qld,qld,1,1,carnegie,carnegie,5163,5163
16,2709,webb,19431014,chevalley loop,NaN,NaN,11.0,14.0,adam,ada,07 20680550,07 20680550,2826430,2826430,vic,vic,10,10,mont albert,mont albert,NaN,NaN
32,9344,badman,19960914,jarrahdale street,NaN,NaN,23.0,23.0,shannon,shannon,02 29205913,02 29205193,6953470,6953470,qld,qld,26,26,bonnells bay,bonnell bay,2257,2257
36,5272,wooley,19530721,trussell place,knackery rhs,knackery rhs,26.0,29.0,dante,dante,NaN,NaN,9656996,9656996,qld,qld,3,3,underdale,undedae,4858,4858
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9936,4026,matthews,19150613,ruthven street,caravn park,caravn park,21.0,21.0,jacobie,jacobue,08 77920512,08 77920512,3246160,3246160,qld,qkd,319,318,ryde,ryd,2567,2567
9962,2052,whale,19160324,deamer crescent,NaN,NaN,NaN,NaN,alana,alnam,07 99388950,07 99288950,1227704,1227604,vic,vic,96,96,yamba,yamba,4350,4350
9964,1900,manson,19330804,muresk street,NaN,NaN,33.0,33.0,noah,noah,02 65493761,02 65493761,9410525,9410525,vic,vci,6,8,elanora heights,elanoraheights,3555,3555


Print number of matches (incomplete set and possibly includes incorrect matches)

In [11]:
print('Exact merge on {0:s} gives {1:d} overlapping records'.format(str(EXACT_COLS), len(exact_merge)))

Exact merge on ['last_name', 'date_of_birth', 'address_1'] gives 905 overlapping records


# Hosein's start
After looking at the data a bit, I would like to introduce some keys based on which I compare these data bases. One can create unique identifiers of each person in this way, but also, I can make sure when I find a match, it is an actual match and not a coincidence based on faulty data input. For example, social security number is supposed (assumption) to be a unique identifier but if we join based on them some values from df_a will have multiple match in df_b.

With inclusion of key concept we can do both exact and fuzzy matches based on the keys. This means we also have more opportunity to use exact matching. 

## Keys:
I will try to include different fields in these keys. The keys are done based on some trials to make sure they are making one to one merges.
- ['first_name','last_name', 'date_of_birth', 'address_1'], which is the given example plus first name since without it, it won't be one to one when matching.
- ['soc_sec_id','first_name'], social security number should be unique, but it's not, so I added first name to make one to one matches
- ['last_name', 'address_2', 'town', 'zipcode','state'], address and last name to make one to one matches.
- ['zipcode','street_number', 'date_of_birth', 'age', 'phone_number'], other address details combined with DoB and phone number (expected to be quite unique already) can make a one to one connection.

We can define more keys, but with this, I have coevered all the fields already.

## Approach

So I understand there are faulty input in the data (typo, abbreviation, etc.). But each record doesn't have this issue in every field. So the idea of multiple keys can help overcome this issue a bit even in case of exact matching. 

Of course fuzzy matching comes to mind for the records with uncleaned data but one idea next to fuzzy matching is to use phoenetic matching. This way we can overcome spelling mistakes. Of course, the keys are defined with some redundancy. So even if, for example, two similar last names are the same in phoenetics, date of birth and address will stop them from matching:

After preparing data a bit, I like to first find exact matches based on these keys and drop them from the original databases before moving to fuzzy matches. 

In each step, after a merge based on exact matching, phoenetic matching, and fuzzy matching, I remove the matched data from the source dfs. I do this to make sure I don't double count the matches and to make the next steps - considering fuzzy matching is slower - less cumbersome. 

## Fuzzy matching

So I read over these different methods a bit. Thanks for the tip about the jellyfish package :)

I started with fuzzy wuzzy package which sorts the words and accounts for different lengths but when it came to actual matching I found it difficult to make a merge without brute-forcing and the fuzzymatch function which is availabel doesn't leave much room for modification on scorer and threshold for finding a match.

So I decided to use fuzzy pandas. It works relatively quick. It takes 4 minutes to make matches based on 4 keys. Fuzzy pandas works with jaro-winkler. I looked at some examples of how different methods will work on the given data:

In [12]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import difflib

In [13]:
#from the given expected match examples
str1 = df_a.loc[[2847]].apply(lambda x: ' '.join(x.dropna().astype(str).values), axis=1).to_string(index=False)
str2 = df_b.loc[[6268]].apply(lambda x: ' '.join(x.dropna().astype(str).values), axis=1).to_string(index=False)
print(fuzz.ratio(str1,str2))
print(fuzz.partial_ratio(str1,str2))
print(fuzz.token_sort_ratio(str1,str2))
print(fuzz.token_set_ratio(str1,str2))
print(jellyfish.jaro_distance(str1,str2))
print(jellyfish.jaro_winkler(str1,str2))
print(difflib.SequenceMatcher(None, str1,str2).ratio())

78
78
70
77
0.8695238095238095
0.9217142857142857
0.76


In [14]:
# the top one was not so good except for jaro-winkler. What if I try it with a specific key?

#from the given expected match examples (this is based on key1)
str1 = df_a[['first_name','last_name', 'date_of_birth', 'address_1']].loc[[2847]].apply(lambda x: ' '.join(x.dropna().astype(str).values), axis=1).to_string(index=False)
str2 = df_b[['first_name','last_name', 'date_of_birth', 'address_1']].loc[[6268]].apply(lambda x: ' '.join(x.dropna().astype(str).values), axis=1).to_string(index=False)
print(fuzz.ratio(str1,str2))
print(fuzz.partial_ratio(str1,str2))
print(fuzz.token_sort_ratio(str1,str2))
print(fuzz.token_set_ratio(str1,str2))
print(jellyfish.jaro_distance(str1,str2))
print(jellyfish.jaro_winkler(str1,str2))
print(difflib.SequenceMatcher(None, str1,str2).ratio())

# great results!

99
97
99
99
0.990990990990991
0.9945945945945945
0.9863013698630136


In [15]:
# Shouldn't be a match! I like to see what will return based on different algorithms here.
str1 = df_a.loc[[2]].apply(lambda x: ' '.join(x.dropna().astype(str).values), axis=1).to_string(index=False)
str2 = df_b.loc[[9344]].apply(lambda x: ' '.join(x.dropna().astype(str).values), axis=1).to_string(index=False)
print(fuzz.ratio(str1,str2))
print(fuzz.partial_ratio(str1,str2))
print(fuzz.token_sort_ratio(str1,str2))
print(fuzz.token_set_ratio(str1,str2))
print(jellyfish.jaro_distance(str1,str2))
print(jellyfish.jaro_winkler(str1,str2))
print(difflib.SequenceMatcher(None, str1,str2).ratio())

40
40
34
34
0.6
0.6
0.26


This is not all the examples that I tried, but just a show case. Jaro-winkler ratio works very well, so for fuzzy pandas I use this method. Threshold of 90 seems a good cut off number based on these examples

First, I will prepare the data, define the keys, merge and remove based on the three mentioned approaches and then discuss what we see and how much is remaining.

In [16]:
# many NaN values cause problems later on during fuzzy matching so they are removed.
df_a = df_a.fillna('')
df_b = df_b.fillna('')

In [17]:
def metaphone_all(df):
    df['last_name_metaphone'] = df['last_name'].apply(lambda x: jellyfish.metaphone(str(x)))
    df['address_1_metaphone'] = df['address_1'].apply(lambda x: jellyfish.metaphone(str(x)))
    df['first_name_metaphone'] = df['first_name'].apply(lambda x: jellyfish.metaphone(str(x)))
    df['address_2_metaphone'] = df['address_2'].apply(lambda x: jellyfish.metaphone(str(x)))
    df['town_metaphone'] = df['town'].apply(lambda x: jellyfish.metaphone(str(x)))
    df['state_metaphone'] = df['state'].apply(lambda x: jellyfish.metaphone(str(x)))
    return(df)
df_a = metaphone_all(df_a)
df_b = metaphone_all(df_b)

In [18]:
# keys defined for exact matches
key1 = ['first_name','last_name', 'date_of_birth', 'address_1']
key2 = ['soc_sec_id','first_name']
key3 = ['last_name', 'address_2', 'town', 'zipcode','state']
key4 = ['zipcode','street_number', 'age', 'phone_number']

In [19]:
# similar keys are used for phoenetic matching. Key 4 is just of numbers. Phoenetic matching won't help in that case.
key1_metaphone = ['last_name_metaphone', 'date_of_birth', 'address_1_metaphone']
key2_metaphone = ['soc_sec_id','first_name_metaphone']
key3_metaphone = ['last_name_metaphone', 'address_2_metaphone', 'town_metaphone', 'zipcode','state_metaphone']

In [20]:
def merge_and_remove(df1,df2, key):
    exact_merge = df1[df1[key].notnull().all(axis=1)]\
                  .reset_index(drop=False)\
                  .merge(df2[df_b[key].notnull().all(axis=1)].reset_index(drop=False),
                         how='inner', on=key, suffixes=('_a', '_b'))\
                  .set_index(['index_a', 'index_b'] , drop=True)
    exact_merge = exact_merge[sorted(exact_merge.columns)]
    # making sure the merge is one to one, also you can see how much is matched in each step
    print(len(exact_merge.index.get_level_values('index_a').unique()))
    print(len(exact_merge.index.get_level_values('index_b').unique()))
    # removing it from the source dfs. This way we won't report these matches twice:
    df1 = df1.drop(exact_merge.index.get_level_values('index_a'))
    df2 = df2.drop(exact_merge.index.get_level_values('index_b'))
    return df1, df2, exact_merge


In [21]:
# Exact matches:
df_a, df_b, exact_merge1 = merge_and_remove(df_a, df_b, key1)
df_a, df_b, exact_merge2 = merge_and_remove(df_a, df_b, key2)
df_a, df_b, exact_merge3 = merge_and_remove(df_a, df_b, key3)
df_a, df_b, exact_merge4 = merge_and_remove(df_a, df_b, key4)


618
618
2060
2060
609
609
876
876


In [22]:
# phoenetic matching:
df_a, df_b, phoenetic_merge1 = merge_and_remove(df_a, df_b, key1_metaphone)
df_a, df_b, phoenetic_merge2 = merge_and_remove(df_a, df_b, key2_metaphone)
df_a, df_b, phoenetic_merge3 = merge_and_remove(df_a, df_b, key3_metaphone)

314
314
66
66
14
14


In [23]:
print(len(df_a))
print(len(df_b))

5443
5443


Now we can see that we have mathced more than 40% of data and less than 40% is remaining to deal with in fuzzy matching.
## Fuzzy matching

Using Jaro-Winkler approach.

In [24]:
import fuzzy_pandas as fpd
import time

In [25]:
def fuzzy_match(df1, df2, key):
    
    # I had to add suffixes since it doesn't do it on it's own like pandas
    keya = [k + '_a' for k in key]
    keyb = [k + '_b' for k in key]
    
    df1['index'] = df1.index
    df2['index'] = df2.index
    
    result = fpd.fuzzy_merge(df1.add_suffix('_a'), df2.add_suffix('_b'),
                             left_on=keya, right_on=keyb, method='jaro', threshold=0.9)
    result = result[sorted(result.columns)]
    
    # making sure the merge is one to one
    print(len(result['index_a'].unique()))
    print(len(result['index_b'].unique()))
    
     # removing it from the source dfs. This way we won't report these matches twice:
    df1 = df1.drop(result['index_a'])
    df2 = df2.drop(result['index_b'])
    return df1, df2, result


In [26]:
tic = time.perf_counter()

df_a, df_b, fuzzy_match1 = fuzzy_match(df_a, df_b, key1)
df_a, df_b, fuzzy_match3 = fuzzy_match(df_a, df_b, key3)
df_a, df_b, fuzzy_match4 = fuzzy_match(df_a, df_b, key4)

# df_a, df_b, fuzzy_match2 = fuzzy_match(df_a, df_b, key2) #I had to remove it since it wasn't a 1to1 match.
# Also I suspect only using two fields, one of which is numeric, doesn't wrok that great in fuzzy matching.

toc = time.perf_counter()
total_time = (toc - tic) /60

print(f"Fuzzy merger is done in {total_time:0.4f} minutes")

108
108
32
32
55
55
Fuzzy merger is done in 3.2557 minutes


In [27]:
fuzzy_match4.head(10)

,address_1_a,address_1_b,address_1_metaphone_a,address_1_metaphone_b,address_2_a,address_2_b,address_2_metaphone_a,address_2_metaphone_b,age_a,age_b,date_of_birth_a,date_of_birth_b,first_name_a,first_name_b,first_name_metaphone_a,first_name_metaphone_b,index_a,index_b,last_name_a,last_name_b,last_name_metaphone_a,last_name_metaphone_b,phone_number_a,phone_number_b,soc_sec_id_a,soc_sec_id_b,state_a,state_b,state_metaphone_a,state_metaphone_b,street_number_a,street_number_b,town_a,town_b,town_metaphone_a,town_metaphone_b,zipcode_a,zipcode_b
0,eagle circuit,eagle circuit,EKL SRKT,EKL SRKT,,,,,33.0,33.0,19351203,19351203,natalia,tillre,NTL,TLR,3,3808,tiller,natalia,TLR,NTL,07 91923807,07 91928307,1245778,1245778,nt,nt,NT,NT,73,73,camden,camden,KMTN,KMTN,5114,5114
1,,,,,,,,,22.0,22.0,19140202,19140202,tiarna,noble,XRN,NBL,35,6774,noble,tiafna,NBL,XFN,04 13060444,04 13066444,8394368,8394368,nsw,nsw,NS,NS,4,4,dromana,dromana,TRMN,TRMN,3193,3193
2,namatjira drive,namatjura drive,NMTJR TRF,NMTJR TRF,cluan,cluan,KLN,KLN,10.0,10.0,19150726,19150726,tayah,taah,TY,T,40,5142,chetter,,XTR,,03 15816045,03 15816045,6877207,6877207,nsw,nsw,NS,NS,29,29,leongatha,leongatha,LNK0,LNK0,3074,3704
3,mckinlay street,mckinlay street,MKNL STRT,MKNL STRT,rose hall,rose hall,RS HL,RS HL,12.0,21.0,,,emiily,emiily,EML,EML,708,7528,heilmair,heil mair,HLMR,HL MR,04 76668873,04 76668873,9341497,9341479,qld,qld,KLT,KLT,39,39,east fremantle,mount hutton,EST FRMNTL,MNT HTN,6530,6530
4,serpentine street,serpentine street,SRPNTN STRT,SRPNTN STRT,,,,,33.0,33.0,19700206,19700206,ella,ellys,EL,ELS,755,9966,van der klaauw,van dercklaauw,FN TR KL,FN TRKL,04 13128865,04 13228865,7342075,7342075,nsw,nsw,NS,NS,4,4,bundaberg,bundaberg,BNTBRK,BNTBRK,4480,4480
5,tiptree crescent,tiptree cteacent,TPTR KRSSNT,TPTR KTSNT,,,,,33.0,33.0,19151126,19151126,timothy,t.,TM0,T,760,4402,coleman,coleman,KLMN,KLMN,04 11666151,04 11666151,9958445,9958445,nsw,nsw,NS,NS,97,97,bagdad,bagdnd,BKTT,BKTNT,2122,2212
6,crozier circuit,crozier circuit,KRSR SRKT,KRSR SRKT,,,,,25.0,25.0,19750109,19750109,olivia,browne,OLF,BRN,804,6437,browne,olivka,BRN,OLFK,02 54392816,02 54392816,6571454,6571454,nsw,nsw,NS,NS,24,243,maitland,maitland,MTLNT,MTLNT,6026,6026
7,broadbent street,broadbent street,BRTBNT STRT,BRTBNT STRT,,,,,8.0,8.0,19420818,19830607,timothy,timotly,TM0,TMTL,988,3961,bradshaw,bradshaw,BRTX,BRTX,02 85292690,02 85292690,1199692,1199692,qld,qld,KLT,KLT,54,54,naremburn,naremburn,NRMBRN,NRMBRN,4153,4135
8,sherwin place,,XRWN PLS,,oak valley,oak valley,OK FL,OK FL,26.0,26.0,19070620,19070620,roisin,roisin,RSN,RSN,1134,2477,eyles,eyles,ELS,ELS,03 78637475,03 78637475,1452398,1452938,nsw,,NS,,55,55,warracknabeal,warracknabeal,WRKNBL,WRKNBL,2536,2563
9,,,,,,,,,34.0,34.0,,,samantha,samantha,SMN0,SMN0,1269,7835,djamatatradja,djamatltradja,TJMTTRTJ,TJMTLTRTJ,08 40158143,08 40154143,2765174,2765417,vic,vic,FK,FK,13,13,kurrajong heights,kurrajong heights,KRJNK HTS,KRJNK HTS,2114,2114


In [28]:
phoenetic_merge3.head(10)

,,address_1_a,address_1_b,address_1_metaphone_a,address_1_metaphone_b,address_2_a,address_2_b,address_2_metaphone,age_a,age_b,date_of_birth_a,date_of_birth_b,first_name_a,first_name_b,first_name_metaphone_a,first_name_metaphone_b,last_name_a,last_name_b,last_name_metaphone,phone_number_a,phone_number_b,soc_sec_id_a,soc_sec_id_b,state_a,state_b,state_metaphone,street_number_a,street_number_b,town_a,town_b,town_metaphone,zipcode
index_a,index_b,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1785,2557,barr-smith avenue,barr-smith avenue,BRSM0 AFN,BRSM0 AFN,,,,37.0,37.0,19911016,,gus,g.,KS,K,white,white,WT,02 23546025,02 23546025,7754138,7754138,,,,2,25,wendouree,wenduree,WNTR,4355
2785,2825,burkitt street,burkitt sgtreet,BRKT STRT,BRKT SKTRT,rosedale,rosedale,RSTL,34.0,34.0,19711203,19711203,ella,ella,EL,EL,fatigati,fatigti,FTKT,08 82827351,08 82823751,7375776,7375676,qld,qld,KLT,150,150,dapto,dapto,TPT,4879
3034,2730,dandenong court,dandenong court,TNTNNK KRT,TNTNNK KRT,brumby's run,brumby's run,BRMBS RN,28.0,29.0,19840228,19840318,rachael,r.,RXL,R,sherriff,sherriff,XRF,04 11098319,04 11098319,7753397,7753397,sa,ss,S,,,bundaberg,bundaberg,BNTBRK,3131
3691,8220,glenorchy street,glenorchyrstreet,KLNRX STRT,KLNRXRSTRT,parkway centre,parkway centre,PRKW SNTR,7.0,7.0,,,karli,k.,KRL,K,miles,miles,MLS,02 16905799,02 16905799,2513061,2512061,vic,vic,FK,29,20,terrigal,terrgal,TRKL,6224
4188,4713,biddell place,biddell qtlace,BTL PLS,BTL KTLS,,,,30.0,30.0,19551207,19551207,dante,d.,TNT,T,caranci,caranci,KRNS,08 74567638,08 74567638,7742112,7742112,nsw,nsw,NS,2,1,glen alice,glenn alice,KLN ALS,6530
5482,7746,kanangra court,court kanangra,KNNKR KRT,KRT KNNKR,,,,34.0,34.0,19260813,19260813,samantha,samantya,SMN0,SMNTY,coleman,colman,KLMN,08 13933897,08 19333897,6542942,6542942,vic,vic,FK,128,128,deer park,deer park,TR PRK,3162
5705,9520,kingsford smith drive,kingsfordnsmiqh drive,KNKSFRT SM0 TRF,KNKSFRTNSMK TRF,,,,27.0,,19860427,19860427,jasmine,j.,JSMN,J,mearns,mearns,MRNS,03 36451646,03 36451646,9124413,9124413,vic,vic,FK,13,13,vaucluse,vauclusse,FKLS,3199
6465,4688,mcconnel crescent,mcconnel crescent,MKKNL KRSSNT,MKKNL KRSSNT,,,,35.0,35.0,19940227,19942027,taylah,t.,TL,T,brock,brocu,BRK,07 65397903,07 65397903,2620783,2620783,nsw,nsw,NS,181,1913,tarragindi,tarragindi,TRJNT,6152
7033,5157,ellerston avenue,ellerston avenue,ELRSTN AFN,ELRSTN AFN,,,,30.0,30.0,19511008,19514008,joel,joeo,JL,J,pennell,penneol,PNL,03 81212125,03 81212125,3933876,3933876,nsw,nsw,NS,3,4,beverly hills,beverly hills,BFRL HLS,3240


In [29]:
print(len(df_a))
print(len(df_b))

5248
5248


Looking at some different matches from these three steps it appears to be correoct matches, since you can see same value for other fields aside from what they have been matched on. But at the same time, this means my approach may have been a bit conservative. Now it depends on the application of data which is best; conservative and accurate or more inclusive and less accurate?! So just because this has been conservative it doesn't mean wrong.

Nonetheless, you can define more keys, even conservatively, that make different combinations of fields. This can help in finding more matches between these two data sets. I also went with the assumption that the relationship between the tables are one to one, in case that is not true, the keys that I used can become somewhat more relaxed.

It seems short in here now, but I've tried many different things, specially in fuzzy matching part. So I don't have much more time to spend and try with new keys as well.

At the end, we managed to find 4752 matches. Respectively, 4163, 394, and 195 are exact, phoenitic, and fuzzy matches. I started with exact merging, and I removed each match before going to the next step, so it is expected that this number decreases step by step. This also points out to the question of effort vs. accuracy which is again a metter of context.